In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 25 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

11 April 2023 ; 13:30:45


In [3]:
scrape_year = 2023

# Data preparation

In [4]:
# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv').groupby('roy').get_group(0) # Load up-to-date data, omit rookies
# Load big data file (to compute careers and comparison)
final_data = pd.read_csv(path + 'final_data.csv')
final_data = final_data[final_data['Year'] < scrape_year]
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
1,Bobby Portis,PF,27,MIL,70,22,26.0,5.7,11.5,0.496,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1
2,Brook Lopez,C,34,MIL,78,78,30.4,6.1,11.5,0.531,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,0
3,Giannis Antetokounmpo,PF,28,MIL,63,63,32.1,11.2,20.3,0.553,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,0
4,Goran Dragić,PG,36,MIL,58,0,15.0,2.4,5.8,0.421,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1
5,Grayson Allen,SG,27,MIL,72,70,27.4,3.4,7.7,0.440,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,0


In [5]:
# For each player, determine his career lifetime, his career high in points, assists, rebounds and minutes played
# as well as his previous playing year
def years_in_league(df):
    player = list(df['Player'])[0] # get player name
    temp = final_data.groupby('Player').get_group(player).sort_values('Year') # get player data sorted by year
    #if len(list(temp['Year'])) < 2 : # if it's his first year set dummy previous year (easy to cut)
      #  df['prev_year'] = [0]
    #else : # else save previous playing year
    df['prev_year'] = [list(temp['Year'])[-1]]

    df['lifetime'] = [len(temp) + 1] # compute career lifetime
    
    df_prev = temp[temp['Year'] < scrape_year] # get previous data
    # Compute career high
    df['PTS_high'] = [df_prev['PTS'].max()]
    df['AST_high'] = [df_prev['AST'].max()]
    df['TRB_high'] = [df_prev['TRB'].max()]
    df['MP_high'] = [df_prev['MP'].max()]

    return df

data = data.groupby('Player').apply(years_in_league)
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRtg/A,NRtg/A,roy,smoy,prev_year,lifetime,PTS_high,AST_high,TRB_high,MP_high
1,Bobby Portis,PF,27,MIL,70,22,26.0,5.7,11.5,0.496,...,112.64,3.48,0,1,2022,8,14.6,1.7,9.1,28.2
2,Brook Lopez,C,34,MIL,78,78,30.4,6.1,11.5,0.531,...,112.64,3.48,0,0,2022,15,20.7,2.3,8.6,36.9
3,Giannis Antetokounmpo,PF,28,MIL,63,63,32.1,11.2,20.3,0.553,...,112.64,3.48,0,0,2022,10,29.9,5.9,13.6,36.7
4,Goran Dragić,PG,36,MIL,58,0,15.0,2.4,5.8,0.421,...,112.64,3.48,0,1,2022,15,20.3,7.4,4.1,35.1
5,Grayson Allen,SG,27,MIL,72,70,27.4,3.4,7.7,0.440,...,112.64,3.48,0,0,2022,5,11.1,2.2,3.4,27.3


In [6]:
# Only keep players having played in 2022 and merge with last year data : may be a case where MIP might be relative to a further year !!!
data = data.groupby('prev_year').get_group(scrape_year-1) 
data = data.merge(final_data.rename(columns = {'Year' : 'prev_year'}), on = ['Player', 'prev_year'], suffixes = ['', '_prev'], how = 'inner')

### FEATURES ####
# Calculate the difference in stats for all numerical features
features =  ['G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST',
             'STL','BLK','TOV','PF','PTS','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%',
             'OWS','DWS','WS','WS/48','OBPM','DBPM','BPM','VORP','W','L','W/L%','PS/G','PA/G','SRS','Seed','MOV','ORtg','DRtg',
             'NRtg','MOV/A','ORtg/A','DRtg/A','NRtg/A','ORtg/100','DRtg/100']
for f in features :
    data[f'{f}_diff'] = data[f] - data[f'{f}_prev']

# Additional features
data['PTS_rel'] = [(pts-prev)/prev if prev > 0 else 0 for pts, prev in zip(data['PTS'], data['PTS_prev'])] # Points relative to previous year
data['PER_rel'] = (data['PER'] - data['PER_prev']) / data['PER_prev'] # PER relative to previous year
data['PTS_high_diff'] = data['PTS'] - data['PTS_high'] # Points difference with career high

### CUTS ###
# Previous seasons requirements (minutes and game)
data = data[data['MP_prev'] >= 5] # Minimum 5 minutes played in previous season
data = data[data['G_prev'] >= 10] # Minimum 10 games played in previous season
# Eligibility
data = data[data['PTS'] > data['PTS_high']] # Career high in points
data = data[(data['AST'] >= data['AST_high']) | (data['TRB'] >= data['TRB_high']) | (data['MP'] >= data['MP_high'])] # At least a career high in minutes, assists or rebounds

data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,NRtg_diff,MOV/A_diff,ORtg/A_diff,DRtg/A_diff,NRtg/A_diff,ORtg/100_diff,DRtg/100_diff,PTS_rel,PER_rel,PTS_high_diff
6,Jevon Carter,PG,27,MIL,81,39,22.3,3.0,7.0,0.423,...,0.18,0.39,0.26,-0.03,0.30,-29.0,-1.0,0.904762,0.135417,3.1
10,Lindell Wigginton,PG,24,MIL,7,1,12.4,2.6,5.3,0.486,...,0.18,0.39,0.26,-0.03,0.30,-1.0,3.0,0.690476,0.156863,2.9
22,Josh Green,SG,22,DAL,60,21,25.7,3.4,6.4,0.537,...,-3.49,-3.26,3.78,7.29,-3.51,5.0,10.0,0.895833,-0.058824,4.3
26,Luka Dončić,PG,23,DAL,66,66,36.2,10.9,22.0,0.496,...,-3.49,-3.26,3.78,7.29,-3.51,10.0,6.0,0.140845,0.143426,3.6
40,Jalen Johnson,SF,21,ATL,70,6,14.9,2.3,4.6,0.491,...,-1.29,-1.23,0.24,1.50,-1.26,16.0,-1.0,1.333333,0.247788,3.2


# Features, model

In [7]:
features = ['PTS_high','FG','VORP','FGA','OWS','WS','MP','PER','TRB','AST','OBPM','USG%']
features = [f + '_diff' for f in features] + ['PER_rel','lifetime']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [8]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mip/mip_data.csv')

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15

In [9]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [10]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1)) # Add rank
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'Tm'] + features + ['Share']]
res.head(5)

,Player,Tm,PTS_high_diff,FG_diff,VORP_diff,FGA_diff,OWS_diff,WS_diff,MP_diff,PER_diff,TRB_diff,AST_diff,OBPM_diff,USG%_diff,PER_rel,lifetime,Share
0,Tyrese Haliburton,IND,5.4,1.8,3.342857,3.2,4.171429,4.128571,-1.4,5.4,-0.3,2.2,4.3,4.8,0.296703,3,0.399652
1,Lauri Markkanen,UTA,6.9,3.6,3.000000,5.8,5.127273,5.187879,3.6,7.4,2.9,0.6,4.5,7.1,0.503401,6,0.295844
2,Shai Gilgeous-Alexander,OKC,6.9,1.9,4.052941,1.5,7.329412,9.147059,0.8,6.3,-0.2,-0.4,2.9,2.0,0.301435,5,0.247243
3,Jalen Brunson,NYK,7.7,2.2,2.320588,4.8,3.420588,2.991176,3.1,4.1,-0.4,1.4,3.5,5.3,0.239766,5,0.187904
4,Alperen Şengün,HOU,5.2,2.4,1.677333,3.4,2.917333,3.585333,8.2,3.7,3.5,1.3,2.7,-0.6,0.231250,2,0.154938


In [11]:
res.to_csv(path + f'Results/mip/results_week_{week}.csv', index = None)